<a href="https://colab.research.google.com/github/DhruvAgg10/Python-projects/blob/main/fakevsrealnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

id: unique id of news article
title: title of news article
author: author of news article
text: text of article could be incomplete
label: tells if news is fake or not

In [5]:
import pandas as pd
import numpy as np
# we need to segregate words and remove comman words which doesnot help in classifying news and false or notso for that import some libs
import re
# regular expression (regex) module, which provides powerful tools for pattern matching and text processing.
#It allows you to search, match, replace, and split strings using patterns.
import nltk
#NLTK (Natural Language Toolkit) is a popular Python library for Natural Language Processing (NLP).
# It provides tools for text processing, including tokenization, stemming, lemmatization, stopword removal
#, part-of-speech tagging, named entity recognition, and more.
from nltk.corpus import stopwords
#Purpose: Removes common words (e.g., "is", "the", "and", "a") that do not contribute much meaning to text analysis.
from nltk.stem.porter import PorterStemmer
#Purpose: Reduces words to their root (stem) form (e.g., "running" → "run", "flies" → "fli"
from sklearn.feature_extraction.text import TfidfVectorizer
#Use Case: Converts text into machine-learning-friendly format.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#basic classification prob therefore using logistic regression
from sklearn.metrics import accuracy_score

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

importing data directly from kaggle to google colab

In [8]:
! pip install opendatasets

In [12]:
import opendatasets as od
od.download('https://www.kaggle.com/c/fake-news/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dhruvaggarwal2
Your Kaggle Key: ··········


100%|██████████| 46.5M/46.5M [00:02<00:00, 18.6MB/s]



Extracting archive ./fake-news/fake-news.zip to ./fake-news


In [13]:
train_dataset=pd.read_csv('/content/fake-news/train.csv')

In [14]:
train_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


LABEL includes whether news is fake or not

*   1:fake
*   0:real



In [15]:
#check if there are any missing values
train_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [16]:
#compared to our dataset missing values are not too much therefore we can replace them with null string
# and still train our ml model effectively
# code to fill null string in place of misisng values
train_dataset=train_dataset.fillna('')
train_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [32]:
# WE WANT TO TRAIN OUR MODEL ONLY ON AUTHOR NAME AND NEWS TITLE SO MERGE THEM IN ONE COLUMN USING
#we didnt include text because each of the text is a long para so it will be taking long time for training
train_dataset['column']=train_dataset['author']+''+train_dataset['title']

In [18]:
#seperate data and label
X= train_dataset.drop(columns='label', axis=1)
Y=train_dataset['label']

In [29]:
# now we have to stem the words to their root words
stem= PorterStemmer()
def stemcontent(content):
  stem_content= re.sub('[^a-zA-Z]',' ',content)
  #Uses re.sub() (regular expressions) to replace any character that is not a letter (i.e., digits, punctuation, and special characters) with an empty string.
  stem_content=stem_content.lower()
  #Converts all text to lowercase to maintain uniformity and avoid case-sensitive mismatches.
  stem_content=stem_content.split()
  #Splits the text into a list of individual words based on spaces.
  stem_content=[stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
  #stem word to their root word which is not included in stopwords
  stem_content=' '.join(stem_content)
  #Joins the list of stemmed words into a single string with spaces separating them.
  return stem_content

In [30]:
train_dataset['column']= train_dataset['column'].apply(stemcontent)

In [31]:
print(train_dataset['column'])

0        darrelllucushousedemaidewedidntevenseecomeysle...
1        danieljflynnflynnhillaryclintonbigwomanoncampu...
2               consortiumnewscomwhythetruthmightgetyoufir
3        jessicapurkisscivilianskilledinsingleusairstri...
4        howardportnoyiranianwomanjailedforfictionalunp...
                               ...                        
20795    jeromehudsonrappertitrumpaposterchildforwhites...
20796    benjaminhoffmannflplayoffsschedulematchupsando...
20797    michaeljdelamercedandrachelabramsmacysissaidto...
20798    alexansarynatorussiatoholdparallelexercisesinb...
20799                            davidswansonwhatkeepsthef
Name: column, Length: 20800, dtype: object


In [67]:
X=train_dataset['column'].values
Y=train_dataset['label'].values
print(X)

['Darrell LucusHouse Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'
 'Daniel J. FlynnFLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
 'Consortiumnews.comWhy the Truth Might Get You Fired' ...
 'Michael J. de la Merced and Rachel AbramsMacy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times'
 'Alex AnsaryNATO, Russia To Hold Parallel Exercises In Balkans'
 'David SwansonWhat Keeps the F-35 Alive']


In [68]:
#TFIDF VECTORIZER DOES : IT IDENTIFIES HOW MANY TIMES A PARTICULAR WORD IS REPEATED AND ACCORDINGLY ASSIGNS IMPORTANCE VALUE TO IT
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X= vectorizer.transform(X)
print(X)

  (0, 1181)	0.26093253981852516
  (0, 5558)	0.3015240959717515
  (0, 6517)	0.20565002646673405
  (0, 7840)	0.2951614509491496
  (0, 8241)	0.2740085657861192
  (0, 8620)	0.24398932179297475
  (0, 10594)	0.23028973074454998
  (0, 16295)	0.1536111400920685
  (0, 16508)	0.20476648736259792
  (0, 18475)	0.24826788911182338
  (0, 19040)	0.35769014842767877
  (0, 28544)	0.22370120836039492
  (0, 33092)	0.34354025525372645
  (0, 33460)	0.2620405036866357
  (0, 34402)	0.17918739747269477
  (1, 3621)	0.3493832273397924
  (1, 4420)	0.18290583060889654
  (1, 5116)	0.4508844821928066
  (1, 6219)	0.23118052033472167
  (1, 7777)	0.313608220062674
  (1, 11652)	0.5156618392052476
  (1, 14631)	0.23703745360274664
  (1, 22730)	0.19046856538729842
  (1, 35080)	0.3619575823267221
  (2, 6696)	0.5074493780278307
  :	:
  (20797, 20519)	0.16984739136130164
  (20797, 21898)	0.07697953798217144
  (20797, 25706)	0.23832116836576178
  (20797, 26083)	0.30196639491652494
  (20797, 27797)	0.23692575764592924
  (20797

TRAIN THE MODEL


In [36]:
model=LogisticRegression()

In [37]:
model.fit(X,Y)

LogisticRegression()

In [38]:
#evaluate training model accuracy
train_prediction=model.predict(X)
train_accuracy=accuracy_score(Y,train_prediction)
print(train_accuracy)

0.9740865384615385


In [80]:
#import test data and sol
test_data=pd.read_csv('/content/fake-news/test.csv')
test_data=test_data.fillna('')


In [81]:
#now create a function to return the test data in perff form
def function(dataset):
  dataset['column']= dataset['author']+' '+dataset['title']
  dataset['column']=dataset['column'].apply(stemcontent)
  X=dataset['column'].values
  X= vectorizer.transform(X)
  return X



In [82]:
test_data=function(test_data)
print(test_data)

  (0, 7894)	0.2347192186625122
  (0, 18939)	0.43791990877734616
  (0, 21898)	0.09500878536003027
  (0, 29361)	0.3801208880768011
  (0, 31111)	0.43791990877734616
  (0, 31149)	0.43791990877734616
  (0, 32357)	0.25456282422137133
  (0, 32932)	0.1293256634342876
  (0, 33630)	0.3535259171549303
  (0, 35459)	0.09776940847414413
  (1, 1313)	0.3541962802785967
  (1, 21744)	0.3878483427652039
  (1, 27631)	0.3217731738039829
  (1, 31144)	0.39164346802911476
  (1, 32110)	0.39164346802911476
  (1, 34313)	0.5601887811902859
  (2, 1509)	0.22871488641212528
  (2, 6589)	0.3017250135582896
  (2, 9306)	0.3254561223459419
  (2, 11312)	0.3673873127546425
  (2, 18246)	0.3236135821714107
  (2, 18262)	0.26525978580445286
  (2, 22184)	0.3625051671990122
  (2, 30633)	0.32015738706458147
  (2, 34060)	0.3361432237020626
  :	:
  (5196, 29092)	0.27519858962216714
  (5196, 29336)	0.2512218632172717
  (5196, 30984)	0.30784564009816306
  (5196, 32357)	0.20388670539093973
  (5196, 33776)	0.3205538172009411
  (5196, 3

In [83]:
#so test data values are in form of correct value
y_ans=pd.read_csv('/content/fake-news/submit.csv')
y_ans=y_ans['label'].values
print(y_ans)

[0 1 0 ... 0 1 0]


In [85]:
#check accuracy
test_prediction=model.predict(test_data)
test_accuracy=accuracy_score(y_ans,test_prediction)
print(test_accuracy)

0.6675
